In [1]:
import pandas as pd
from amplpy import AMPL, Environment

In [17]:
ampl = AMPL(Environment('/opt/ampl.linux64'))

In [18]:
ampl.reset()

In [19]:
print(ampl.getOption('solver'))
ampl.setOption('solver','cplex')
print(ampl.getOption('solver'))

minos
cplex


In [20]:
model="""
#PART1: DECISION VARIABLES/UNKNOWN VARIABLES
var x1 >= 0;
var x2 >= 0;

#PART2: OBJECTIVE FUNCTION
maximize objfunc: x1 + x2;

#PART3: CONSTRAINTS
s.t. c1: x1 + 2*x2 <= 4;
s.t. c2: 4*x1 + 2*x2 <= 12;
s.t. c3: -x1 + x2 <= 1;

#Each constraint has a name. That's how we will refer to each constraint.
#The objective function has a name.
#Notice c3 is an equality constraint
"""

In [21]:
ampl.eval(model)

In [22]:
print(ampl.getObjective('objfunc').get())
ampl.eval('expand objfunc;')

maximize objfunc:
	x1 + x2;
maximize objfunc:
	x1 + x2;



In [23]:
for i in ampl.getConstraints():
    print(i[1])
#print(ampl.getConstraint('c1'))
#print(ampl.getConstraint('c2'))
#print(ampl.getConstraint('c3'))

subject to c1: x1 + 2*x2 <= 4;
subject to c2: 4*x1 + 2*x2 <= 12;
subject to c3: -x1 + x2 <= 1;


In [24]:
ampl.eval('expand c1;')
ampl.eval('expand c2;')
ampl.eval('expand c3;')

subject to c1:
	x1 + 2*x2 <= 4;

subject to c2:
	4*x1 + 2*x2 <= 12;

subject to c3:
	-x1 + x2 <= 1;



In [25]:
ampl.eval('expand x1;')
ampl.eval('expand x2;')

Coefficients of x1:
	c1        1
	c2        4
	c3       -1
	objfunc   1

Coefficients of x2:
	c1       2
	c2       2
	c3       1
	objfunc  1



In [26]:
ampl.eval('display _nvars, _ncons;')

_nvars = 2
_ncons = 3



In [27]:
ampl.solve()

CPLEX 12.8.0.0: optimal solution; objective 3.333333333
2 dual simplex iterations (1 in phase I)


In [28]:
print(ampl.getObjective('objfunc').get().value())

3.333333333333333


#### Shadow Price or Dual Price or Marginal Price

In [29]:
for i in ampl.getConstraints():
    print(i[1].getValues().toPandas())

    c1.dual
0  0.333333
    c2.dual
0  0.166667
   c3.dual
0      0.0


In [30]:
for i in ampl.getVariables():
    print(i[1].getValues())

   x1.val   
2.6666666666666665

   x2.val   
0.66666666666666663



### Decision Variable Solution and Reduced Cost

In [31]:
X_result = list()
for v in ampl.getVariables():
    X_result.append([
        v[0],
        v[1].lb(), 
        v[1].value(),
        v[1].ub(),
        v[1].rc()
    ])
df_X_result = pd.DataFrame(X_result)
df_X_result.rename(columns={0:'DecisionVariable_X',
                            1:'X_LowerBound',
                            2:'X_Solution',
                            3:'X_UpperBound',
                            4:'X_ReducedCosts'},
                   inplace=True)
df_X_result.set_index('DecisionVariable_X',inplace=True)
df_X_result

,X_LowerBound,X_Solution,X_UpperBound,X_ReducedCosts
DecisionVariable_X,,,,
x1,0.0,2.666667,inf,0.0
x2,0.0,0.666667,inf,0.0
